# IMPORTING ALL LIBRARIES

In [1]:
import numpy
import tensorflow as tf
import pandas
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.utils import shuffle

Using TensorFlow backend.


#  fix random seed for reproducibility

In [2]:
seed = 7
numpy.random.seed(seed)

# load dataset

In [6]:
dataframe = pandas.read_csv('sonar.csv', header= None)
dataset = dataframe.values
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

X[0].shape

(60,)

In [7]:
    encoder_Y = preprocessing.LabelEncoder()
    encoder_Y.fit(Y)
LabelEncoder()

LabelEncoder()

In [8]:
encoder_Y.classes_

array(['M', 'R'], dtype=object)

In [9]:
encoder_Y.transform(Y)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# baseline model

In [12]:

# baseline model
def create_baseline():
    model = Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,)))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model, write code below
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

# evaluate model with standardized dataset

In [13]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Results: 86.04% (6.33%)


# evaluate baseline model with standardized dataset

In [14]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 84.59% (3.67%)


# smaller model

In [15]:
def create_smaller():
    model = Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,)))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model, write code below
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [16]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller: 83.59% (6.26%)


# larger model

In [12]:
def larger_model():
    model = Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,)))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [13]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=larger_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 84.11% (7.12%)


# functional model

In [14]:
def functional_model():
    inputs = keras.Input(shape=(60,))
    layer1 = Dense(60, activation='relu')(inputs)
    layer2 = Dense(60, activation='relu')(layer1)
    outputs = Dense(1, activation='sigmoid')(layer2)

    model = keras.Model(inputs, outputs)
    
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [15]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=functional_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Functional Api: Accuracy %.2f%%, Error (%.2f%%)" % (results.mean()*100, results.std()*100))

Functional Api: Accuracy 83.59%, Error (4.01%)


# Rewriting the code using the Keras Functional API

In [16]:
from keras.layers import Input
from keras.layers import Dense
from keras.utils import plot_model
from keras.models import Model

Inputs = Input(shape=(60,))
x = Dense(60, activation='relu')(Inputs)
x = Dense(60, activation='relu')(x)
x = Dense(60, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=Inputs, outputs=output)

print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 60)                0         
_________________________________________________________________
dense_161 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_162 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_163 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_164 (Dense)            (None, 1)                 61        
Total params: 11,041
Trainable params: 11,041
Non-trainable params: 0
_________________________________________________________________
None


# Rewriting the code by doing Model Subclassing

In [18]:
import tensorflow as tf

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = Dense(60, activation = 'relu')
        self.dense2 = Dense(60, activation = 'relu')
        self.dense3 = Dense(1, activation = 'sigmoid')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)
    
model = MyModel()

# rewriting tha code without using scikit learn

In [19]:
dataframe = shuffle(dataframe)

dataset = dataframe.values

X = dataset[:, 0:60].astype(float)
y = dataset[:, 60]

# label encoding
le = LabelEncoder()
encoded_y = le.fit(y).transform(y)

k=10
num_val_samples = len(X)//k
num_epochs = 100

all_acc = numpy.array([])
# all_err = np.array([])

In [20]:
for i in range(k):
    print('Processing K-Fold #', i+1)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded_y[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_data = numpy.concatenate(
        [X[:i * num_val_samples],
         X[(i + 1) * num_val_samples:]],
        axis=0)
    
    partial_train_targets = numpy.concatenate(
        [encoded_y[:i * num_val_samples],
         encoded_y[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = functional_model()
    history = model.fit(partial_train_data, partial_train_targets,
        epochs=num_epochs, batch_size=5, verbose=0, validation_data=(val_data, val_targets))
    
    results = model.evaluate(val_data, val_targets, verbose=0)
    
#     all_err = np.append(all_err, results[0])
    all_acc = numpy.append(all_acc, results[1])
    
print("+--------------------+")    
print("Custom K-Folds: Accuracy %.2f%%, Error (%.2f%%)" % (all_acc.mean()*100, all_acc.std()*100))

Processing K-Fold # 1
Processing K-Fold # 2
Processing K-Fold # 3
Processing K-Fold # 4
Processing K-Fold # 5
Processing K-Fold # 6
Processing K-Fold # 7
Processing K-Fold # 8
Processing K-Fold # 9
Processing K-Fold # 10
+--------------------+
Custom K-Folds: Accuracy 82.50%, Error (6.02%)
